# Convert the Valpo Sounding to NCAR EOL Aspen CSV Input
Follow the specs outlined on the docs:
https://ncar.github.io/aspendocs/man_intro.html

## Imports

In [80]:
import pandas as pd
import glob
from pathlib import Path
from datetime import datetime

## Read the file(s)

In [5]:
files = sorted(glob.glob("*.GPS"))
files

['20240727_05_SHGPS.GPS']

In [69]:
file = files[0]
gps_file = open(file, 'r', encoding='iso-8859-1')
ogdata = gps_file.readlines()

df = pd.read_csv(file,
                 skiprows=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10,],
                 delim_whitespace=True,
                 encoding='iso-8859-1')
# Fix the incorrect column name due to space in Elapsed time
good_columns = df.columns[1:]
df = df.drop(columns="HeightE")
df.columns = good_columns

In [77]:
hour_min = ogdata[2].split("\t")[1][:-1]
day_month = ogdata[3].split("\t")[1][:-1]
time = datetime.strptime(f"{day_month} {hour_min}", "%d/%m/%y %H:%M:%S")

launch_year = str(time.year) # 4 digit year
launch_month = str(time.month).zfill(2) # 2 digit month
launch_day = str(time.day).zfill(2) # 2 digit day
launch_hour = str(time.hour).zfill(2) # 2 digit hour
launch_minute = str(time.minute).zfill(2) # 2 digit minute
launch_second = str(time.second).zfill(2) # 2 digit second

first_ob = df.iloc[0].astype(str)
P = first_ob.P         # initial pressure at launch
T = first_ob.Temp        # initial temperature at launch
RH = first_ob.RH        # inital relative humidity at launch
ws = first_ob.Speed       # initial wind speed at launch
wd = first_ob.Dir      # initial wind direction at launch
lat = first_ob.Lat      # initial latitude at launch
lon = first_ob.Lon     # initial longitude at launch
gps_alt = first_ob.HeightE # initial altitude (geometric height/gps alt)
dew = first_ob.Dewp      # initial dewpoint at launch

In [79]:
# Open new output file
CSV_file = open("test.csv", "w")

# Write ASPEN-formatted header to new file:
CSV_file.write("FileFormat, CSV" + '\n')
CSV_file.write("Year, " + launch_year + '\n')
CSV_file.write("Month, " + launch_month + '\n')
CSV_file.write("Day, " + launch_day + '\n')
CSV_file.write("Hour, " + launch_hour + '\n')
CSV_file.write("Minute, " + launch_minute + '\n')
CSV_file.write("Second, " + launch_second + '\n')
CSV_file.write("Pressure, " + str(P) + ", units=mb " + '\n')
CSV_file.write("Temperature, " + str(T) + ", units=deg " + '\n')
CSV_file.write("RH, " + str(RH) + ", units=% " + '\n')
CSV_file.write("Speed, " + str(ws) + ", units=m/s " + '\n')
CSV_file.write("Direction, " + str(wd) + ", units=deg " + '\n')
CSV_file.write("Latitude, " + str(lat) + ", units=deg " + '\n')
CSV_file.write("Longitude, " + str(lon) + ", units=deg " + '\n')
CSV_file.write("Altitude, " + str(gps_alt) + ", units=m " + '\n')  # initial GPS altitude
CSV_file.write("Dewpoint, " + str(dew) + ", units=deg " + '\n')
CSV_file.write("Fields,Time,Pressure,Temperature,RH,Speed,Direction,Latitude,Longitude,Altitude,GPSAlt,Dewpoint" + '\n')
CSV_file.write("Units,sec,mb,deg C,%,m/s,deg,deg,deg,m,m,deg C" + '\n')

# Write ASPEN-formatted data to new file
# NOTE: Missing data with "/" present as filler should be just blank/empty in CSV file
# according to ASPEN documentation
newdata = []
for row in range(len(df)):
    newdata = df.iloc[row].astype(str)
    CSV_file.write("Data," + newdata["time"] + ',')     # Time (sec)
    if '/' not in newdata[1]:
        CSV_file.write(newdata["P"] + ',')           # Pressure (mb)
    else:
        CSV_file.write(',')
    if '/' not in newdata["Temp"]:
        CSV_file.write(newdata["Temp"] + ',')           # Temperature (deg. C)
    else:
        CSV_file.write(',')
    if '/' not in newdata["RH"]:
        CSV_file.write(newdata["RH"] + ',')           # Relative Humidity (%) (U [%] in GRAW's rts files)
    else:
        CSV_file.write(',')
    if '/' not in newdata["Speed"]:
        CSV_file.write(newdata["Speed"] + ',')          # Wind Speed (m/s)
    else:
        CSV_file.write(',')
    if '/' not in newdata["Dir"]:
        CSV_file.write(newdata["Dir"] + ',')          # Wind Direction (deg.)
    else:
         CSV_file.write(',')
    if '/' not in newdata["Lat"]:
        CSV_file.write(newdata["Lat"] + ',')          # Latitude (deg.)
    else:
         CSV_file.write(',')
    if '/' not in newdata["Lon"]:
        CSV_file.write(newdata["Lon"] + ',')          # Longitude (deg.)
    else:
         CSV_file.write(',')
    if '/' not in newdata["HeightE"]:
        CSV_file.write(newdata["HeightE"] + ',')           # Altitude/Geopotential Height (m)
    else:
         CSV_file.write(',')
    if '/' not in newdata["HeightMSL"]:
        CSV_file.write(newdata["HeightMSL"] + ',')          # GPS Altitude/Geometric Height (m)
    else:
         CSV_file.write(',')
    if '/' not in newdata["Dewp"]:
        CSV_file.write(newdata["Dewp"] + '\n')          # Dewpoint (deg. C)
    else:
         CSV_file.write('\n')

# Close CSV data file
CSV_file.close()
print('Processing File '+str(x+1)+' of '+str(len(text_files))+' COMPLETE!')
print(' ')

/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/ipykernel_47843/1708446406.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if '/' not in newdata[1]:
/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/ipykernel_47843/1708446406.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if '/' not in newdata[1]:
/var/folders/bw/c9j8z20x45s2y20vv6528qjc0000gq/T/ipykernel_47843/1708446406.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

NameError: name 'x' is not defined

In [30]:

from datetime import datetime
now = datetime.now()
formatted = now.strftime("%d/%m/%Y %H:%M:%S")
print(formatted)

01/08/2024 09:42:19


In [ ]:
### NAME:  convert_rts2csv_GRAW5.16.04.22.py

### MODIFICATION HISTORY:
#   Believed to have originally been written by Courtney, then modified by 
#   Alycia Gilliland for GRAINEX and RELAMPAGO (2019).
#   
#   Updated by Maiana Hanshaw (03/28/2020):
#       1. Updated code to import/output Geometric Height (GPS Altitude) in addition to 
#          Geopotential Height
#       2. Added code to be able to handle rts/csv files from all vehicles (DOWs, SCOUTs, & UIUCs)
#       3. Added code to import hours, minutes and seconds from rts file and not file name,
#          so this no longer needs manually updating.
#
#   Updated by Paul Robinson (08/04/2022):
#		1.	Changed input path to current directory "./"
#		2.  Changed input file wildcard to "rts.txt" so all .rts files in current directory run.
#		3.	Fixed header units row by adding "m" in "GPSAlt" column.
#
#   Updated by Josh Aikins (08/31/2022):
#       1.  Made code so that input filename is not case sensitive ('rts.txt' or 'RTS.txt' work)
#       2.  Made code so that script does not need to be in same directory as files
#       3.  Now outputs blank/empty data for missing data rather than the strange "///" strings
#           that show up in the RTS data files, since ASPEN expects blank data for missing
#       4.  Cleaned up code (removed not used features, renamed variables, added comments)
#
#   Updated by Josh Aikins (09/05/2023):
#       1. Adapted from convert_rts2csv_GRAW5.14.10.3.py to work with new GRAWMET 5.16.04.22
#       2. Added print statements so user can detect issues while running code

### PURPOSE:
#   To read in raw "RTS" data files produced by GRAWMET (version 5.16.04.22) and output into "CSV"
#   format so that they can be run through ASPEN. See online documentation here:
#       https://ncar.github.io/aspendocs/man_input.html
#   
#   RTS data filename convention must match the following, where <VID> is the vehicle ID or name 
#   and dates/times are referencing the launch time:
#       YYYYMMDD_HHMM_IOPXX_<VID>_rts.txt

###################################################################################################

import os  # operating system library

###### UPDATE THIS ######
## Add rts text files to a working directory, then specify the path to the files
#path = "P:/Edits/sounding_QC/scripts/rts_test"
#path ="./"
path = "P:/Edits/20230216_IOP01/SONDE1/soundings/20230713/2-L0"
#########################

print('Processing Directory: '+path)
text_files = [f for f in os.listdir(path) if f.endswith(('rts.txt','RTS.txt'))]
print('Found '+str(len(text_files))+' RTS Data Files!')
print(' ')

for x in range(0, len(text_files)):
    # Extract file name
    filename = text_files[x]
    print('Processing File '+str(x+1)+' of '+str(len(text_files))+': '+filename)
    
    # Open RTS file and read each line:
    RTS_file = open(path+'/'+filename, 'r')
    ogdata = RTS_file.readlines()
    
    # Close RTS file
    RTS_file.close()
    
    # Extract each line of data (skip header line) and split based on tab delimiter
    array = []
    for i in range(1, len(ogdata)):
        array.append(ogdata[i]r.split('\t'))
    
    # Grab first line of data (launch surface conditions)
    sfc_obs = array[0]
    
    # Get the actual hour, minute, second data from the rts initial utc launch time
    utc = float(sfc_obs[15])
    
    h = utc/3600
    hh = int(h)
        
    m = (h - hh) * 60
    mm = int(m)
        
    s = (m - mm) * 60
    ss = round(s)
    
    if ss <= 59:
        hour = hh
        minute = mm
        second = ss

    new_mm = 0
    new_hh = 0
    if ss == 60:
        if mm <= 59:
            new_mm = mm + 1
            if new_mm <= 59:
                hour = hh
                minute = new_mm
                second = 0
            if new_mm == 60:
                new_hh == hh + 1
                if new_hh <= 23:
                    hour = new_hh
                    minute = 0
                    second = 0
                if new_hh == 24:
                    hour = 0
                    minute = 0
                    second = 0

    if hour < 10:            
        launch_hh = "0" + str(hour)
    else:
        launch_hh = str(hour)
        
    if minute < 10:
        launch_mm = "0" + str(minute)
    else:
        launch_mm = str(minute)
        
    if second < 10:
        launch_ss = "0" + str(second)
    else:
        launch_ss = str(second)
        
    #########################

    # Get year, month, date information from the file name, but adjust if launch time is a new day:
    year = int(filename[0:4])
    month = int(filename[4:6])
    day = int(filename[6:8])  

    month_day_options = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    
    if hour == 0 and minute == 0 and second == 0:
        days_in_month = month_day_options[month]
        if day < days_in_month:
            new_day = day + 1
            year_int = year
            month_int = month
            day_int = new_day
        if day == days_in_month:
            new_month = month + 1
            year_int = year
            month_int = new_month
            day_int = 1
            if new_month <= 12:
                year_int = year
                month_int = new_month
                day_int = 1
            if new_month == 13:
                year_int = year + 1
                month_int = 1
                day_int = 1
                
        launch_year = str(year_int)
        if month_int < 10:
            launch_month = "0" + str(month_int)
        else:
            launch_month = str(month_int)
        if day_int < 10:
            launch_day = "0" + str(day_int)
        else:
            launch_day = str(day_int)
               
    else:
        launch_year = filename[0:4]
        launch_month = filename[4:6]
        launch_day = filename[6:8]
    
    print('Launch Time = '+launch_month+'/'+launch_day+'/'+launch_year+' '+launch_hh+':'+launch_mm+':'+launch_ss+' UTC')
    print(' ')
    
    #########################
    
    # Retrieve launch surface observations
    P = sfc_obs[1]         # initial pressure at launch
    T = sfc_obs[2]         # initial temperature at launch
    RH = sfc_obs[3]        # inital relative humidity at launch
    ws = sfc_obs[16]       # initial wind speed at launch
    wd = sfc_obs[17]       # initial wind direction at launch
    lat = sfc_obs[22]      # initial latitude at launch
    lon = sfc_obs[21]      # initial longitude at launch
    gps_alt = sfc_obs[23]  # initial altitude (geometric height/gps alt)
    dew = sfc_obs[5]       # initial dewpoint at launch
    
    print('Surface Observations Summary:')
    print("  Latitude, " + str(lat) + " deg ")
    print("  Longitude, " + str(lon) + " deg ")
    print("  Altitude, " + str(gps_alt) + " m MSL ")
    print("  Pressure, " + str(P) + " mb ")
    print("  Temperature, " + str(T) + " degC ")
    print("  Dewpoint, " + str(dew) + " degC ")
    print("  RH, " + str(RH) + " % ")
    print("  Wind Speed, " + str(ws) + " m/s ")
    print("  Wind Direction, " + str(wd) + " degrees from N ")
    print(' ')
    
    #########################
    
    # Create new output file name
    #outfile = filename.replace("_rts.txt",".csv")
    outfile = filename[:-8]+'.csv'
    print('Outputting Data to CSV File: '+outfile)
    
    # Open new output file
    CSV_file = open(path+'/'+outfile, "w")
    
    # Write ASPEN-formatted header to new file:
    CSV_file.write("FileFormat, CSV" + '\n')
    CSV_file.write("Year, " + launch_year + '\n')
    CSV_file.write("Month, " + launch_month + '\n')
    CSV_file.write("Day, " + launch_day + '\n')
    CSV_file.write("Hour, " + launch_hh + '\n')
    CSV_file.write("Minute, " + launch_mm + '\n')
    CSV_file.write("Second, " + launch_ss + '\n')
    CSV_file.write("Pressure, " + str(P) + ", units=mb " + '\n')
    CSV_file.write("Temperature, " + str(T) + ", units=deg " + '\n')
    CSV_file.write("RH, " + str(RH) + ", units=% " + '\n')
    CSV_file.write("Speed, " + str(ws) + ", units=m/s " + '\n')
    CSV_file.write("Direction, " + str(wd) + ", units=deg " + '\n')
    CSV_file.write("Latitude, " + str(lat) + ", units=deg " + '\n')
    CSV_file.write("Longitude, " + str(lon) + ", units=deg " + '\n')
    CSV_file.write("Altitude, " + str(gps_alt) + ", units=m " + '\n')  # initial GPS altitude
    CSV_file.write("Dewpoint, " + str(dew) + ", units=deg " + '\n')
    CSV_file.write("Fields,Time,Pressure,Temperature,RH,Speed,Direction,Latitude,Longitude,Altitude,GPSAlt,Dewpoint" + '\n')
    CSV_file.write("Units,sec,mb,deg C,%,m/s,deg,deg,deg,m,m,deg C" + '\n')

    # Write ASPEN-formatted data to new file
    # NOTE: Missing data with "/" present as filler should be just blank/empty in CSV file
    # according to ASPEN documentation
    newdata = []
    for newdata in array:
        CSV_file.write("Data," + newdata[0] + ',')     # Time (sec)
        if '/' not in newdata[1]:
            CSV_file.write(newdata[1] + ',')           # Pressure (mb)
        else:
            CSV_file.write(',')
        if '/' not in newdata[2]:
            CSV_file.write(newdata[2] + ',')           # Temperature (deg. C)
        else:
            CSV_file.write(',')
        if '/' not in newdata[3]:
            CSV_file.write(newdata[3] + ',')           # Relative Humidity (%) (U [%] in GRAW's rts files)
        else:
            CSV_file.write(',')
        if '/' not in newdata[16]:
            CSV_file.write(newdata[16] + ',')          # Wind Speed (m/s)
        else:
            CSV_file.write(',')
        if '/' not in newdata[17]:
            CSV_file.write(newdata[17] + ',')          # Wind Direction (deg.)
        else:
             CSV_file.write(',')
        if '/' not in newdata[22]:
            CSV_file.write(newdata[22] + ',')          # Latitude (deg.)
        else:
             CSV_file.write(',')
        if '/' not in newdata[21]:
            CSV_file.write(newdata[21] + ',')          # Longitude (deg.)
        else:
             CSV_file.write(',')
        if '/' not in newdata[4]:
            CSV_file.write(newdata[4] + ',')           # Altitude/Geopotential Height (m)
        else:
             CSV_file.write(',')
        if '/' not in newdata[23]:
            CSV_file.write(newdata[23] + ',')          # GPS Altitude/Geometric Height (m)
        else:
             CSV_file.write(',')
        if '/' not in newdata[5]:
            CSV_file.write(newdata[5] + '\n')          # Dewpoint (deg. C)
        else:
             CSV_file.write('\n')
    
    # Close CSV data file
    CSV_file.close()
    print('Processing File '+str(x+1)+' of '+str(len(text_files))+' COMPLETE!')
    print(' ')